# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
import gc
import os
import re
from typing import List, Dict

import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs

from sklearn.cluster import KMeans, DBSCAN, MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

# import warnings
# warnings.filterwarnings('ignore')

In [2]:
# load files
# train = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/train.parquet')
# train_y = train['selected']
# train = train.drop('selected')

# data = pl.concat([train, pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/test.parquet')])
data = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/train.parquet').drop('selected')

# del train
gc.collect()
# sample_submission = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/sample_submission.parquet')

64

In [3]:
data.head()

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments2_baggageAllowance_quantity,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,legs1_segments3_aircraft_code,legs1_segments3_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_iata,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType,legs1_segments3_cabinClass,legs1_segments3_departureFrom_airport_iata,legs1_segments3_duration,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,__index_level_0__
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64
0,true,57323,null,"""S7/SU/UT""",36,false,false,"""2024-06-15T16:20:00""","""2024-06-15T15:40:00""","""02:40:00""","""YK2""","""KJA""","""KJA""",1.0,0.0,1.0,"""TLK""","""02:40:00""","""216""","""KV""","""KV""",9.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,370.0,16884.0,0
1,true,57323,123,"""S7/SU/UT""",36,true,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""","""07:25:00""","""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""","""02:50:00""","""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""","""01:20:00""","""5311""","""S7""","""S7""",4.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2300.0,null,1.0,3500.0,null,1.0,1.0,1,2087645,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",true,2240.0,51125.0,1
2,true,57323,null,"""S7/SU/UT""",36,false,false,"""2024-06-15T14:50:00""","""2024-06-15T09:25:00""","""07:25:00""","""E70""","""OVB""","""OVB""",1.0,0.0,1.0,"""TLK""","""02:50:00""","""5358""","""S7""","""S7""",4.0,"""E70""","""KJA""","""KJA""",1.0,0.0,1.0,"""OVB""","""01:20:00""","""5311""","""S7""","""S7""",4.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2300.0,null,1.0,3

In [4]:
data.tail()

Id,bySelf,companyID,corporateTariffCode,frequentFlyer,nationality,isAccess3D,isVip,legs0_arrivalAt,legs0_departureAt,legs0_duration,legs0_segments0_aircraft_code,legs0_segments0_arrivalTo_airport_city_iata,legs0_segments0_arrivalTo_airport_iata,legs0_segments0_baggageAllowance_quantity,legs0_segments0_baggageAllowance_weightMeasurementType,legs0_segments0_cabinClass,legs0_segments0_departureFrom_airport_iata,legs0_segments0_duration,legs0_segments0_flightNumber,legs0_segments0_marketingCarrier_code,legs0_segments0_operatingCarrier_code,legs0_segments0_seatsAvailable,legs0_segments1_aircraft_code,legs0_segments1_arrivalTo_airport_city_iata,legs0_segments1_arrivalTo_airport_iata,legs0_segments1_baggageAllowance_quantity,legs0_segments1_baggageAllowance_weightMeasurementType,legs0_segments1_cabinClass,legs0_segments1_departureFrom_airport_iata,legs0_segments1_duration,legs0_segments1_flightNumber,legs0_segments1_marketingCarrier_code,legs0_segments1_operatingCarrier_code,legs0_segments1_seatsAvailable,legs0_segments2_aircraft_code,legs0_segments2_arrivalTo_airport_city_iata,…,legs1_segments2_baggageAllowance_quantity,legs1_segments2_baggageAllowance_weightMeasurementType,legs1_segments2_cabinClass,legs1_segments2_departureFrom_airport_iata,legs1_segments2_duration,legs1_segments2_flightNumber,legs1_segments2_marketingCarrier_code,legs1_segments2_operatingCarrier_code,legs1_segments2_seatsAvailable,legs1_segments3_aircraft_code,legs1_segments3_arrivalTo_airport_city_iata,legs1_segments3_arrivalTo_airport_iata,legs1_segments3_baggageAllowance_quantity,legs1_segments3_baggageAllowance_weightMeasurementType,legs1_segments3_cabinClass,legs1_segments3_departureFrom_airport_iata,legs1_segments3_duration,legs1_segments3_flightNumber,legs1_segments3_marketingCarrier_code,legs1_segments3_operatingCarrier_code,legs1_segments3_seatsAvailable,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,profileId,ranker_id,requestDate,searchRoute,sex,taxes,totalPrice,__index_level_0__
i64,bool,i64,i64,str,i64,bool,bool,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,str,str,…,f64,f64,f64,str,str,str,str,str,f64,str,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,datetime[ns],str,bool,f64,f64,i64
18146427,true,54154,44,null,36,true,false,"""2024-11-06T09:45:00""","""2024-11-05T20:50:00""","""10:55:00""","""738""","""SGC""","""SGC""",0.0,0.0,1.0,"""VKO""","""03:20:00""","""247""","""UT""","""UT""",3.0,"""AT7""","""SVX""","""SVX""",0.0,0.0,1.0,"""SGC""","""02:30:00""","""111""","""UT""","""UT""",5.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,24000.0,null,1.0,0.0,null,0.0,0.0,1,3046852,"""88f8c53a28bf4f438941fd67338009…",2024-10-29 12:46:20,"""MOWSVX/SVXMOW""",true,5560.0,30730.0,18146427
18146428,true,54154,44,null,36,true,false,"""2024-11-05T20:00:00""","""2024-11-05T00:20:00""","""17:40:00""","""738""","""UFA""","""UFA""",0.0,0.0,1.0,"""VKO""","""02:05:00""","""363""","""UT""","""UT""",3.0,"""AT7""","""SVX""","""SVX""",0.0,0.0,1.0,"""UFA""","""01:30:00""","""106""","""UT""","""UT""",9.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,24000.0,null,1.0,0.0,null,0.0,0.0,1,3046852,"""88f8c53a28bf4f438941fd67338009…",2024-10-29 12:46:20,"""MOWSVX/SVXMOW""",true,5560.0,27660.0,18146428
18146429,true,54154,44,null,36,true,false,"""2024-11-05T20:00:00""","""2024-11-05T00:20:00""","""17:40:00""","""738""","""UFA""","""UFA""",0.0,0.0,1.0,"""VKO""","""02:05:00""","""363""","""UT""","""UT""",3.0,"""AT7""","""SVX""","""SVX""",0.0,0.0,1.0,"""UFA""","""01:30:00""","""106""","""UT""","""UT""",9.0,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,

In [5]:
data.shape

(18145372, 126)

In [5]:
# determine the number of null values for each feature
results = []
for col in data.columns:
    null_count = data.select(pl.col(col).is_null().sum()).item()
    results.append({
        'column': col,
        'data_type': str(data.schema[col]),
        'null_count': null_count,
        'null_percentage': round(null_count / len(data) * 100, 2)
    })
null_summary = pl.DataFrame(results)

In [6]:
# features with no NULL values:
null_summary.filter(pl.col('null_count') == 0)

column,data_type,null_count,null_percentage
str,str,i64,f64
"""Id""","""Int64""",0,0.0
"""bySelf""","""Boolean""",0,0.0
"""companyID""","""Int64""",0,0.0
"""nationality""","""Int64""",0,0.0
"""isAccess3D""","""Boolean""",0,0.0
…,…,…,…
"""searchRoute""","""String""",0,0.0
"""sex""","""Boolean""",0,0.0
"""taxes""","""Float64""",0,0.0


In [7]:
# features with NULL values < 10%
null_summary.filter((pl.col("null_count") > 0) & (pl.col("null_percentage") < 10.0)).sort("null_count")

column,data_type,null_count,null_percentage
str,str,i64,f64
"""legs0_segments0_departureFrom_…","""String""",2,0.0
"""legs0_segments0_arrivalTo_airp…","""String""",6,0.0
"""legs0_segments0_aircraft_code""","""String""",14,0.0
"""legs0_segments0_arrivalTo_airp…","""String""",113,0.0
"""legs0_segments0_baggageAllowan…","""Float64""",1064,0.01
…,…,…,…
"""pricingInfo_isAccessTP""","""Float64""",905045,4.99
"""miniRules0_monetaryAmount""","""Float64""",1395743,7.69
"""miniRules1_monetaryAmount""","""Float64""",1395743,7.69


In [8]:
# features with NULL values 10% - 50%
null_summary.filter((pl.col("null_percentage") >= 10.0) & (pl.col("null_percentage") < 50.0)).sort("null_count")

column,data_type,null_count,null_percentage
str,str,i64,f64
"""legs1_arrivalAt""","""String""",4387201,24.18
"""legs1_departureAt""","""String""",4387201,24.18
"""legs1_duration""","""String""",4387201,24.18
"""legs1_segments0_aircraft_code""","""String""",4387201,24.18
"""legs1_segments0_arrivalTo_airp…","""String""",4387201,24.18
…,…,…,…
"""legs1_segments0_arrivalTo_airp…","""String""",4387237,24.18
"""legs1_segments0_cabinClass""","""Float64""",4525013,24.94
"""legs1_segments0_baggageAllowan…","""Float64""",4527297,24.95


At first glance, I would suspect these nulls indicate one-way tickets. Might need to figure out how to handle return flight selection.

In [9]:
# features with NULL values > 50%
null_summary.filter((pl.col("null_percentage") >= 50.0)).sort("null_count")

column,data_type,null_count,null_percentage
str,str,i64,f64
"""corporateTariffCode""","""Int64""",9233925,50.89
"""frequentFlyer""","""String""",12012727,66.2
"""legs0_segments1_arrivalTo_airp…","""String""",14311665,78.87
"""legs0_segments1_departureFrom_…","""String""",14311665,78.87
"""legs0_segments1_duration""","""String""",14311665,78.87
…,…,…,…
"""legs1_segments3_duration""","""String""",18145366,100.0
"""legs1_segments3_flightNumber""","""String""",18145366,100.0
"""legs1_segments3_marketingCarri…","""String""",18145366,100.0


These features are missing a large percentage of values. Given the description provided, we will do the following:
* corporateTariffCode (Corporate tariff code for businesss travel policies): impute to 0 (zero)
* frequentFlyer (Frequent flyer program status): impute to 'NONE'
* miniRules0_percentage (Percentage penalty for cancellation): impute to 0.0 (zero)
* miniRules1_percentage (Percentage penalty for exchange): impute to 0.0 (zero)
* legs[0,1]_segments[1-3]_* (flight segments): evaluate whether to ignore all but segment 1 or impute from segment 1

In [5]:
# check how many unique search sessions there are
print(f"unique sessions (ranker_id): {data['ranker_id'].unique().count()}")

unique sessions (ranker_id): 105539


In [6]:
# check how many unique customers
print(f"unique customers: {data['profileId'].unique().count()}")

unique customers: 32922


In [7]:
# let's look at carrier codes
unique_carrier_codes = (
    data.select(cs.contains("Carrier_code"))
    .unpivot()
    .select("value")
    .filter(pl.col("value").is_not_null())
    .unique().to_series().to_list()
)
print(unique_carrier_codes)

['WK', 'BL', '2C', 'UL', '7C', 'EN', 'EW', 'NI', 'AD', 'UX', 'J2', 'KN', 'VZ', 'T7', '4R', 'СИ', 'LJ', 'AI', 'YK', 'S7', 'AT', 'AH', 'OO', 'OS', 'PI', '9F', 'OK', 'AM', 'W2', 'LX', 'OU', 'BA', 'SU', 'ME', 'R6', 'UO', 'OD', 'I2', 'DL', 'TU', 'OM', 'KB', 'CM', 'AA', 'DP', 'SQ', 'UK', 'NP', 'OZ', 'UT', 'SZ', '4Z', 'ИК', 'AY', '0V', 'ТГ', 'SK', 'OV', 'KQ', 'A5', 'I5', 'DV', 'GS', 'CZ', 'JM', 'JD', 'R3', 'QF', 'QR', 'GM', 'KP', '3U', 'A9', 'WB', 'V7', 'LA', 'KE', 'ZF', 'IQ', 'GW', 'LG', 'HZ', 'HX', 'ZN', 'YC', '3L', 'VN', 'VJ', 'EI', 'MQ', '8M', 'UA', 'ЮЭ', 'D2', 'MU', 'LO', 'MF', '9B', 'TR', '5N', '2P', 'H9', 'CA', 'VA', '5F', 'MS', 'CI', 'TW', 'KA', 'SA', 'IZ', 'SN', 'YX', 'SV', 'WS', '7G', 'IB', 'U6', '6H', '4G', 'PW', 'RE', 'BG', 'HO', 'GQ', 'AZ', 'EY', 'IO', 'D8', 'JL', 'FV', 'CJ', 'YU', 'V3', 'HV', 'GF', 'GA', 'WZ', 'VF', 'СЛ', 'B2', 'AV', 'PK', 'TG', 'BT', 'ET', 'B6', '7T', 'K6', 'Y7', '6E', 'F6', 'PC', '2L', 'KC', 'NX', 'CD', 'NL', 'LY', 'RO', 'HU', 'KU', 'FZ', 'TK', 'ZH', 'BI', 'BR

In [ ]:
carrier_cols = [col for col in data.columns if "Carrier_code" in col]
unique_carrier_codes = (
    data
    .select(
        pl.concat_list([pl.col(col) for col in carrier_cols])
        .alias("carrier_codes")
    )
    .select(
        pl.col("carrier_codes")
        .list.explode()
        .filter(pl.col("carrier_codes").is_not_null())
        .unique()
    )
    .to_series()
    .to_list()
)
print(unique_carrier_codes)

In [25]:
data['frequentFlyer'].unique()

frequentFlyer
str
"""SU/U6/S7"""
"""- ЮТэйр ЗАО"""
"""SU/UT/S7/DL/LH"""
"""SU/DP/S7/UT"""
"""SU/S7/UT"""
…
"""UT/S7/U6/SU"""
"""SU/AI"""
"""KC/C7/SU"""


Frequent flyer code '- ЮТэйр ЗАО' translates to 'UTair JSC', which has no matching carrier under carrier codes.

In [27]:
# create a list of the unique frequent flyer codes
unique_ff_codes = (
    data.select("frequentFlyer")
    .filter(pl.col("frequentFlyer").is_not_null())  # Remove nulls
    .with_columns(
        pl.col("frequentFlyer").str.split("/").alias("ff_codes")  # Split by "/"
    )
    .select("ff_codes")
    .explode("ff_codes")  # Flatten the lists into individual rows
    .unique()  # Get unique codes
    .sort("ff_codes")  # Optional: sort them
    .to_series()
    .to_list()
)
print(unique_ff_codes)

# let's see if any do not have an associated carrier code
ff_not_in_carrier = set(unique_ff_codes) - set(unique_carrier_codes)
print("FF codes not in carrier list:", ff_not_in_carrier)


['- ЮТэйр ЗАО', '2G', '5N', '6R', '6W', '9X', 'A3', 'A4', 'AA', 'AB', 'AF', 'AI', 'AM', 'AT', 'AY', 'AZ', 'B2', 'BA', 'BD', 'C7', 'CA', 'CX', 'CZ', 'D2', 'D9', 'DL', 'DP', 'DV', 'EK', 'ET', 'EY', 'FV', 'FZ', 'GF', 'HU', 'HY', 'I8', 'J2', 'JP', 'JU', 'KC', 'KE', 'KL', 'LH', 'LO', 'LX', 'MS', 'MU', 'N4', 'OK', 'OS', 'OZ', 'PY', 'QR', 'R3', 'RT', 'S7', 'SK', 'SN', 'SQ', 'SU', 'SV', 'TK', 'U6', 'UA', 'UN', 'UT', 'VS', 'WY', 'Y7']
FF codes not in carrier list: {'AB', 'D9', '2G', 'UN', '- ЮТэйр ЗАО', 'JP', '6W', 'VS', 'BD', 'C7', 'PY', '9X'}


Frequent Flyer codes to Airlines:
* ЮТэйр ЗАО (translates to 'UTair JSC'):  UTair / UT
* AB: Air Berlin / AB
* D9: Daallo Airlines Somalia / D9
* 2G: Northwest Seaplanes / None
* UN: Transaero Airlines / UN (no longer in service)
* JP: Adria Airways / JP (no longer in service)
* 6W: Unknown
* VS: Virgin Atlantic Airways / VS
* BD: BMI British Midland / BD (integrated into British Airways)
* C7: Cinnamon Air / C7
* PY: Surinam Airways / PY
* 9X: Air Columbia / Unknown

So, translate 'ЮТэйр ЗАО' to UT and we'll drop any that don't match an airline IATA code

In [15]:
iata_cols = [col for col in data.columns if "_iata" in col]

unique_iata_codes = (
    data
    .select([
        pl.concat_list([pl.col(col) for col in iata_cols])
        .list.explode()
        .drop_nulls()
        .unique()
        .alias('unique_iata_codes')
    ])
    .get_column('unique_iata_codes')
    .to_list()
)

print(f"Found {len(unique_iata_codes)} unique IATA codes")
print(unique_iata_codes)


Found 679 unique IATA codes
['STW', 'PNH', 'BWN', 'TIF', 'LZY', 'TJM', 'MOW', 'AUH', 'GES', 'CGN', 'OMS', 'SSH', 'EYK', 'YUL', 'DQA', 'CRL', 'XYD', 'LLV', 'KYZ', 'ASB', 'DOY', 'DUR', 'VAN', 'MAD', 'JMK', 'CZL', 'NAL', 'XIY', 'QDU', 'KVX', 'LUN', 'KHV', 'UYN', 'KRK', 'VCP', 'KSQ', 'NOZ', 'ANK', 'ASF', 'KWI', 'IST', 'ORY', 'TMJ', 'NNM', 'NYC', 'EAP', 'SEZ', 'KCY', 'CCU', 'MBA', 'BKO', 'GOX', 'KZN', 'XWC', 'BPE', 'IWA', 'SKG', 'BUS', 'URC', 'LJG', 'BSL', 'VRA', 'NAP', 'HHL', 'BRE', 'BQT', 'ESL', 'VOG', 'ALG', 'QIN', 'DAD', 'IRT', 'KNY', 'BUR', 'GRZ', 'PRG', 'HLD', 'SGN', 'SJW', 'XVQ', 'ADB', 'SHA', 'DYG', 'HOU', 'VRN', 'BLQ', 'OLZ', 'WAW', 'WHA', 'AER', 'PNR', 'HFE', 'KRO', 'NCE', 'JHB', 'HYN', 'PYJ', 'BAH', 'LOS', 'PKV', 'ZVJ', 'SYX', 'SEL', 'WNZ', 'BAK', 'UKX', 'MUC', 'NZG', 'YTO', 'KHH', 'PHL', 'ZDN', 'UCT', 'BZX', 'SHE', 'HMA', 'TSA', 'CIT', 'AGY', 'JGS', 'VNO', 'KUF', 'SCO', 'RAK', 'VCE', 'ZFQ', 'SIN', 'DMS', 'CGR', 'WVB', 'ATL', 'TAS', 'DUB', 'ZYR', 'SZB', 'HSN', 'UGC', 'TYD', 'ADA'

Clean-up
1.

In [13]:
# Variables/Lists used for flight feature engineering and extraction
CUSTOMER_FEATURES = ['profileId', 'companyID', 'sex', 'nationality', 'bySelf']
POLARS_INDEX_COL = ['__index_level_0__']
UNUSED_FLIGHT_FEATURES = [
    'legs0_arrivalAt', 'legs0_departureAt', 'legs1_arrivalAt', 'legs1_departureAt',
    'legs0_duration', 'legs1_duration', 'searchRoute',
    'isVip', 'isAccess3D', 'frequentFlyer', 'ff_normalized',
    r'^legs[01]_segments.*$'
]
MAJOR_HUBS = [
    'ATL','DXB','DFW','HND','LHR','DEN','ORD','IST','PVG','ICN','CDG', 'JFK','CLT','MEX','SFO','EWR','MIA','BKK','GRU','HKG'
]


# function to get column groups for the flight segments features that will be used in engineering the new features
def get_column_groups(df: pl.DataFrame) -> Dict[str, List[str]]:
    columns = df.columns
    return {
        'leg0_duration': [col for col in columns if col.startswith('legs0_segments') and col.endswith('_duration')],
        'leg1_duration': [col for col in columns if col.startswith('legs1_segments') and col.endswith('_duration')],
        'mkt_carrier': [col for col in columns if col.endswith('marketingCarrier_code')],
        'op_carrier': [col for col in columns if col.endswith('operatingCarrier_code')],
        'aircraft': [col for col in columns if col.endswith('aircraft_code')],
        'airport': [col for col in columns if 'airport_iata' in col or 'airport_city_iata' in col],
        'seats_avail': [col for col in columns if col.endswith('seatsAvailable')],
        'baggage_type': [col for col in columns if 'baggageAllowance_weightMeasurementType' in col],
        'baggage_allowance': [col for col in columns if 'baggageAllowance_quantity' in col],
        'cabin_class': [col for col in columns if 'cabinClass' in col]
    }


## Creating functions that return Polars expressions, so the feature engineering can be modularized and performed 'lazy' ##
def parse_duration_to_minutes(duration_col: str) -> pl.Expr:
    """Parse duration string to minutes (handles format like '1.02:30:00')."""
    return (
        pl.when(pl.col(duration_col).is_not_null())
        .then(
            pl.col(duration_col).str.extract(r'^(?:(\d+)\.)?', 1).fill_null('0').cast(pl.Int32) * 1440 +
            pl.col(duration_col).str.extract(r'(\d+):(\d+):(\d+)$', 1).fill_null('0').cast(pl.Int32) * 60 +
            pl.col(duration_col).str.extract(r'(\d+):(\d+):(\d+)$', 2).fill_null('0').cast(pl.Int32) +
            (pl.col(duration_col).str.extract(r'(\d+):(\d+):(\d+)$', 3).fill_null('0').cast(pl.Int32) / 60).round(0)
        )
        .otherwise(0)
    )


def create_basic_features(col_groups: Dict[str, List[str]]) -> List[pl.Expr]:
    """Create basic customer and route characteristics features."""
    return [
        # Customer flag features to boolean
        pl.col('isVip').cast(pl.Int32).alias('is_vip'),
        pl.col('isAccess3D').cast(pl.Int32).alias('is_access3D'),

        # Normalized frequentFlyer program, addressing null values as null strings, and translating UT program
        pl.col('frequentFlyer').str.replace('- ЮТэйр ЗАО', 'UT').fill_null('').alias('ff_normalized'),

        # Route characteristics
        pl.col('legs1_departureAt').is_not_null().cast(pl.Int32).alias('is_roundtrip'),
        pl.col('searchRoute').str.slice(0, 3).alias('route_origin'),

        # Hub features
        pl.col('searchRoute').str.slice(0, 3).is_in(MAJOR_HUBS).cast(pl.Int32).alias('origin_is_major_hub'),
        pl.col('searchRoute').str.slice(3, 3).is_in(MAJOR_HUBS).cast(pl.Int32).alias('destination_is_major_hub'),
        pl.max_horizontal([
            pl.col(col).is_in(MAJOR_HUBS) for col in col_groups['airport']
        ]).cast(pl.Int32).alias('includes_major_hub'),
    ]


def create_segment_features(col_groups: Dict[str, List[str]]) -> List[pl.Expr]:
    """Create flight segment related features."""
    return [
        # Number of segments per leg
        pl.concat_list([pl.col(col) for col in col_groups['leg0_duration']])
        .list.drop_nulls().list.len().alias('leg0_num_segments'),

        pl.concat_list([pl.col(col) for col in col_groups['leg1_duration']])
        .list.drop_nulls().list.len().alias('leg1_num_segments'),

        # Total segments
        pl.sum_horizontal([pl.col(col).is_not_null() for col in col_groups['aircraft']])
        .alias('total_segments'),

        # Flight time in minutes (sum of segment durations)
        pl.sum_horizontal([
            pl.col(col).str.extract(r'^(\d+):(\d+)', 1).cast(pl.Int32, strict=False).fill_null(0).mul(60)
            .add(pl.col(col).str.extract(r'^(\d+):(\d+)', 2).cast(pl.Int32, strict=False).fill_null(0))
            for col in col_groups['leg0_duration']
        ]).alias('leg0_flight_time_min'),

        pl.sum_horizontal([
            pl.col(col).str.extract(r'^(\d+):(\d+)', 1).cast(pl.Int32, strict=False).fill_null(0).mul(60)
            .add(pl.col(col).str.extract(r'^(\d+):(\d+)', 2).cast(pl.Int32, strict=False).fill_null(0))
            for col in col_groups['leg1_duration']
        ]).alias('leg1_flight_time_min'),
    ]


def create_time_features() -> List[pl.Expr]:
    """Create time-based features."""
    return [
        # Booking lead time
        ((pl.col('legs0_departureAt').str.to_datetime().cast(pl.Datetime) -
          pl.col('requestDate').cast(pl.Datetime)) / pl.duration(minutes=1))
        .cast(pl.Int32).alias('booking_lead_time_min'),

        # Duration features
        parse_duration_to_minutes('legs0_duration').alias('leg0_duration_min'),
        parse_duration_to_minutes('legs1_duration').alias('leg1_duration_min'),

        # Departure/arrival hours
        pl.col('legs0_departureAt').str.to_datetime().dt.hour().alias('leg0_departure_hour'),
        pl.col('legs0_arrivalAt').str.to_datetime().dt.hour().alias('leg0_arrival_hour'),
        pl.col('legs1_departureAt').str.to_datetime().dt.hour().alias('leg1_departure_hour'),
        pl.col('legs1_arrivalAt').str.to_datetime().dt.hour().alias('leg1_arrival_hour'),
    ]


def create_carrier_features(col_groups: Dict[str, List[str]]) -> List[pl.Expr]:
    """Create carrier and aircraft features."""
    # Create combined carrier column group for carrier features
    all_carrier_cols = col_groups['mkt_carrier'] + col_groups['op_carrier']

    # Create frequent flyer matching expressions
    ff_matches_mkt = [
        pl.when(pl.col(col).is_not_null() & (pl.col(col) != ''))
        .then(pl.col('ff_normalized').str.contains(pl.col(col)))
        .otherwise(False)
        for col in col_groups['mkt_carrier']
    ]
    ff_matches_op = [
        pl.when(pl.col(col).is_not_null() & (pl.col(col) != ''))
        .then(pl.col('ff_normalized').str.contains(pl.col(col)))
        .otherwise(False)
        for col in col_groups['op_carrier']
    ]

    return [
        # Carrier features
        pl.concat_list([pl.col(col) for col in all_carrier_cols])
        .list.drop_nulls().list.unique().list.len().alias('unique_carriers'),

        pl.coalesce([pl.col(col) for col in all_carrier_cols]).alias('primary_carrier'),
        pl.col('legs0_segments0_marketingCarrier_code').alias('marketing_carrier'),

        # Frequent flyer matching (check if FF programs match carriers)
        pl.max_horizontal(ff_matches_mkt).cast(pl.Int32).alias('has_mkt_ff'),
        pl.max_horizontal(ff_matches_op).cast(pl.Int32).alias('has_op_ff'),

        # Aircraft features
        pl.concat_list([pl.col(col) for col in col_groups['aircraft']])
        .list.drop_nulls().list.unique().list.len().alias('aircraft_diversity'),

        pl.coalesce([pl.col(col) for col in col_groups['aircraft']]).alias('primary_aircraft'),
    ]


def create_service_features(col_groups: Dict[str, List[str]]) -> List[pl.Expr]:
    """Create service-related features (fees, cabin, seats, baggage)."""
    return [
        # Cancellation/exchange fees
        (
            ((pl.col('miniRules0_monetaryAmount') > 0) & pl.col('miniRules0_monetaryAmount').is_not_null()) |
            ((pl.col('miniRules0_percentage') > 0) & pl.col('miniRules0_percentage').is_not_null())
        ).cast(pl.Int32).alias('has_cancellation_fee'),

        (
            ((pl.col('miniRules1_monetaryAmount') > 0) & pl.col('miniRules1_monetaryAmount').is_not_null()) |
            ((pl.col('miniRules1_percentage') > 0) & pl.col('miniRules1_percentage').is_not_null())
        ).cast(pl.Int32).alias('has_exchange_fee'),

        # Cabin class features
        pl.max_horizontal([pl.col(col) for col in col_groups['cabin_class']]).alias('max_cabin_class'),
        pl.mean_horizontal([pl.col(col) for col in col_groups['cabin_class']]).alias('avg_cabin_class'),

        # Seat availability (using for understanding popularity/scarcity)
        pl.min_horizontal([pl.col(col) for col in col_groups['seats_avail']]).alias('min_seats_available'),
        pl.sum_horizontal([pl.col(col).fill_null(0) for col in col_groups['seats_avail']]).alias('total_seats_available'),

        # Baggage allowance
        pl.min_horizontal([
            pl.when(pl.col(type_col) == 0).then(pl.col(qty_col)).otherwise(pl.lit(None))
            for type_col, qty_col in zip(col_groups['baggage_type'], col_groups['baggage_allowance'])
        ]).fill_null(0).alias("baggage_allowance_quantity"),

        pl.min_horizontal([
            pl.when(pl.col(type_col) == 1).then(pl.col(qty_col)).otherwise(pl.lit(None))
            for type_col, qty_col in zip(col_groups['baggage_type'], col_groups['baggage_allowance'])
        ]).fill_null(0).alias("baggage_allowance_weight")
    ]


def add_route_popularity(lazy_df: pl.LazyFrame) -> pl.LazyFrame:
    """Add search route popularity features."""
    return lazy_df.with_columns([
        pl.len().over('searchRoute').alias('route_popularity')
    ]).with_columns([
        pl.col('route_popularity').log1p().alias('route_popularity_log')
    ])


def extract_flight_features(df: pl.DataFrame) -> pl.DataFrame:
    """ Aggregate function for processing feature extraction functions"""
    # Determine if dataframe already processed by checking for a calculated feature
    if 'total_duration' in df.columns:
        return df

    # Get column groups
    col_groups = get_column_groups(df)

    # Start with lazy frame, dropping unnecessary columns
    lazy_df = df.drop(CUSTOMER_FEATURES + POLARS_INDEX_COL, strict=False).lazy()

    # Apply all feature groups
    lazy_df = lazy_df.with_columns([
        *create_basic_features(col_groups),
        *create_segment_features(col_groups),
        *create_time_features(),
        *create_service_features(col_groups)
    ])

    # Add carrier features (requires ff_normalized from create_basic_features() step)
    lazy_df = lazy_df.with_columns(create_carrier_features(col_groups))

    # Add total duration (depends on leg durations calculated in create_time_features() above)
    lazy_df = lazy_df.with_columns([
        pl.when(pl.col('leg1_duration_min').is_null())
        .then(pl.col('leg0_duration_min'))
        .otherwise(pl.col('leg0_duration_min') + pl.col('leg1_duration_min'))
        .alias('total_duration')
    ])

    # Add route popularity
    lazy_df = add_route_popularity(lazy_df)

    # Materialize to generate new features, drop unused features, and fill nulls
    return (lazy_df
            .select(pl.exclude(UNUSED_FLIGHT_FEATURES))
            .fill_null(0)
            .collect())


In [14]:
data = extract_flight_features(data)

In [15]:
data.head(300)

Id,corporateTariffCode,frequentFlyer,isVip,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,ranker_id,requestDate,taxes,totalPrice,is_vip,is_access3D,ff_normalized,is_roundtrip,route_origin,origin_is_major_hub,destination_is_major_hub,includes_major_hub,leg0_num_segments,leg1_num_segments,total_segments,leg0_flight_time_min,leg1_flight_time_min,booking_lead_time_min,leg0_duration_min,leg1_duration_min,leg0_departure_hour,leg0_arrival_hour,leg1_departure_hour,leg1_arrival_hour,has_cancellation_fee,has_exchange_fee,max_cabin_class,avg_cabin_class,min_seats_available,total_seats_available,baggage_allowance_quantity,baggage_allowance_weight,unique_carriers,primary_carrier,marketing_carrier,has_mkt_ff,has_op_ff,aircraft_diversity,primary_aircraft,total_duration,route_popularity,route_popularity_log
i64,i64,str,bool,f64,f64,f64,f64,f64,f64,f64,i64,str,datetime[ns],f64,f64,i32,i32,str,i32,str,i32,i32,i32,u32,u32,u32,i32,i32,i32,f64,f64,i8,i8,i8,i8,i32,i32,f64,f64,f64,f64,f64,f64,u32,str,str,i32,i32,u32,str,f64,u32,f64
0,0,"""S7/SU/UT""",false,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,370.0,16884.0,0,0,"""S7/SU/UT""",1,"""TLK""",0,0,0,1,1,2,160,155,42516,160.0,155.0,15,16,9,14,0,0,1.0,1.0,9.0,18.0,1.0,0.0,1,"""KV""","""KV""",0,0,1,"""YK2""",315.0,25,3.258097
1,123,"""S7/SU/UT""",false,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,51125.0,0,1,"""S7/SU/UT""",1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,1,1,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097
2,0,"""S7/SU/UT""",false,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,53695.0,0,0,"""S7/SU/UT""",1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,1,1,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097
3,123,"""S7/SU/UT""",false,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,81880.0,0,1,"""S7/SU/UT""",1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,0,0,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097
4,0,"""S7/SU/UT""",false,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,2240.0,86070.0,0,0,"""S7/SU/UT""",1,"""TLK""",0,0,0,2,2,4,250,245,42141,445.0,505.0,9,14,22,8,0,0,1.0,1.0,4.0,26.0,1.0,0.0,1,"""S7""","""S7""",1,1,1,"""E70""",950.0,25,3.258097
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
295,0,null,false,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""0e51002eda914a39a9309cecf0a659…",2024-05-17 04:47:52,720.0,7420.0,0,0,"""""",0,"""OVB""",0,0,0,1,0,1,80,0,22372,80.0,0.0,17,19,0,0,1,1,1.0,1.0,9.0,9.0,1.0,0.0,1,"""S7""","""S7""",0,0,1,"""E70""",80.0,4522,8.416931
296,115,null,false,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1,"""0e51002eda914a39a9309cecf0a659…",2024-05-17 04:47:52,720.0,17040.0,0,1,"""""",0,"""OVB""",0,0,0,1,0,1,80,0,22372,80.0,0.0,17,19,0,0,0,0,1.0,1.0,9.0,9.0,1.0,0.0,1,"""S7""","""S7""",0,0,1,"""E70""",80.0,4522,8.416931
297,0,null,false,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1,"""0e51002eda914a39a9309cecf0a659…",2024-05-17 04:47:52,720.0,18080.0,0,0,"""""",0,"""OVB""",0,0,0,1,0,1,80,0,22372,80.0,0.0,17,19,0,0,0,0,1.0,1.0,9.0,9.0,1.0,0.0,1,"""S7""","""S7""",0,0,1,"""E70""",80.0,4522,8.416931


In [17]:
data.filter(pl.col('includes_major_hub') == 1).limit(20)

Id,corporateTariffCode,frequentFlyer,isVip,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,ranker_id,requestDate,taxes,totalPrice,is_vip,is_access3D,ff_normalized,is_roundtrip,route_origin,origin_is_major_hub,destination_is_major_hub,includes_major_hub,leg0_num_segments,leg1_num_segments,total_segments,leg0_flight_time_min,leg1_flight_time_min,booking_lead_time_min,leg0_duration_min,leg1_duration_min,leg0_departure_hour,leg0_arrival_hour,leg1_departure_hour,leg1_arrival_hour,has_cancellation_fee,has_exchange_fee,max_cabin_class,avg_cabin_class,min_seats_available,total_seats_available,baggage_allowance_quantity,baggage_allowance_weight,unique_carriers,primary_carrier,marketing_carrier,has_mkt_ff,has_op_ff,aircraft_diversity,primary_aircraft,total_duration,route_popularity,route_popularity_log
i64,i64,str,bool,f64,f64,f64,f64,f64,f64,f64,i64,str,datetime[ns],f64,f64,i32,i32,str,i32,str,i32,i32,i32,u32,u32,u32,i32,i32,i32,f64,f64,i8,i8,i8,i8,i32,i32,f64,f64,f64,f64,f64,f64,u32,str,str,i32,i32,u32,str,f64,u32,f64
2742,0,null,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,"""23e784fb9fad452591eea495fdec14…",2024-05-17 08:00:43,550.0,17480.0,0,0,"""""",1,"""MOW""",0,1,1,1,1,2,310,275,5749,310.0,275.0,7,13,13,18,0,0,1.0,1.0,5.0,14.0,0.0,0.0,1,"""DP""","""DP""",0,0,1,"""73H""",585.0,248307,12.422425
2743,0,null,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,"""23e784fb9fad452591eea495fdec14…",2024-05-17 08:00:43,550.0,20845.0,0,0,"""""",1,"""MOW""",0,1,1,1,1,2,310,275,5749,310.0,275.0,7,13,13,18,0,0,1.0,1.0,5.0,14.0,1.0,0.0,1,"""DP""","""DP""",0,0,1,"""73H""",585.0,248307,12.422425
2744,0,null,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,"""23e784fb9fad452591eea495fdec14…",2024-05-17 08:00:43,550.0,27085.0,0,0,"""""",1,"""MOW""",0,1,1,1,1,2,310,275,5749,310.0,275.0,7,13,13,18,0,0,1.0,1.0,5.0,14.0,1.0,0.0,1,"""DP""","""DP""",0,0,1,"""73H""",585.0,248307,12.422425
2745,0,null,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,"""23e784fb9fad452591eea495fdec14…",2024-05-17 08:00:43,19449.0,43704.0,0,0,"""""",1,"""MOW""",0,1,1,1,1,2,240,225,6454,240.0,225.0,19,23,9,13,0,0,1.0,1.0,9.0,18.0,0.0,0.0,1,"""TK""","""TK""",0,0,1,"""333""",465.0,248307,12.422425
2746,0,null,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,"""23e784fb9fad452591eea495fdec14…",2024-05-17 08:00:43,19449.0,49644.0,0,0,"""""",1,"""MOW""",0,1,1,1,1,2,240,225,6454,240.0,225.0,19,23,9,13,0,0,1.0,1.0,9.0,18.0,0.0,23.0,1,"""TK""","""TK""",0,0,1,"""333""",465.0,248307,12.422425
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2757,0,null,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,"""23e784fb9fad452591eea495fdec14…",2024-05-17 08:00:43,550.0,17480.0,0,0,"""""",1,"""MOW""",0,1,1,1,1,2,310,285,5749,310.0,285.0,7,13,18,23,0,0,1.0,1.0,3.0,8.0,0.0,0.0,1,"""DP""","""DP""",0,0,1,"""73H""",595.0,248307,12.422425
2758,0,null,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,"""23e784fb9fad452591eea495fdec14…",2024-05-17 08:00:43,550.0,20845.0,0,0,"""""",1,"""MOW""",0,1,1,1,1,2,310,285,5749,310.0,285.0,7,13,18,23,0,0,1.0,1.0,3.0,8.0,1.0,0.0,1,"""DP""","""DP""",0,0,1,"""73H""",595.0,248307,12.422425
2759,0,null,false,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,"""23e784fb9fad452591eea495fdec14…",2024-05-17 08:00:43,550.0,27085.0,0,0,"""""",1,"""MOW""",0,1,1,1,1,2,310,285,5749,310.0,285.0,7,13,18,23,0,0,1.0,1.0,3.0,8.0,1.0,0.0,1,"""DP""","""DP""",0,0,1,"""73H""",595.0,248307,12.422425


In [7]:
def add_search_price_features(df):

    lazy_df = df.lazy().with_columns([
        # Basic ranking and percentile features
        pl.col('totalPrice').rank(method='min').over('ranker_id').alias('price_rank'),

        # Price percentile (0-100, lower = cheaper)
        ((pl.col('totalPrice').rank(method='min').over('ranker_id') - 1) /
         (pl.col('totalPrice').count().over('ranker_id') - 1) * 100)
        .fill_null(50.0).alias('price_percentile'),

        # Z-score within search group
        pl.when(pl.col('totalPrice').std().over('ranker_id') > 0)
        .then(
            (pl.col('totalPrice') - pl.col('totalPrice').mean().over('ranker_id')) /
            pl.col('totalPrice').std().over('ranker_id')
        )
        .otherwise(0.0)
        .alias('price_zscore'),

        # Group statistics for ratio calculations
        pl.col('totalPrice').min().over('ranker_id').alias('_min_price'),
        pl.col('totalPrice').max().over('ranker_id').alias('_max_price'),
        pl.col('totalPrice').median().over('ranker_id').alias('_median_price'),
        pl.col('totalPrice').mean().over('ranker_id').alias('_mean_price'),
        pl.col('totalPrice').std().over('ranker_id').alias('_std_price'),
        pl.col('totalPrice').count().over('ranker_id').alias('options_count'),

    ]).with_columns([
        # Ratio features
        (pl.col('totalPrice') / pl.col('_min_price')).alias('price_ratio_to_min'),
        (pl.col('totalPrice') / pl.col('_max_price')).alias('price_ratio_to_max'),
        (pl.col('totalPrice') / pl.col('_median_price')).alias('price_ratio_to_median'),

        # Binary position features
        (pl.col('totalPrice') == pl.col('_min_price')).cast(pl.Int32).alias('is_cheapest'),
        (pl.col('totalPrice') == pl.col('_max_price')).cast(pl.Int32).alias('is_most_expensive'),
        (pl.col('totalPrice') < pl.col('_median_price')).cast(pl.Int32).alias('is_below_median'),

        # Price range and variability
        (pl.col('_max_price') - pl.col('_min_price')).alias('price_range'),
        pl.col('_std_price').alias('price_std'),

        # Coefficient of variation
        pl.when(pl.col('_mean_price') > 0)
        .then(pl.col('_std_price') / pl.col('_mean_price'))
        .otherwise(0.0)
        .alias('price_cv'),

    ]).with_columns([
        # Sort prices to calculate price gap
        pl.col('totalPrice').sort().over('ranker_id').alias('_sorted_prices'),

    ]).with_columns([
        # Calculate price gap using window functions
        pl.when(pl.col('price_rank') < pl.col('options_count'))
        .then(
            # Get the next higher price in the sorted list
            pl.col('_sorted_prices').shift(-1).over('ranker_id') - pl.col('totalPrice')
        )
        .otherwise(0.0)
        .alias('price_gap_to_next_cheapest')

    ]).drop(['^_.*$']).fill_null(0)  # Remove the temporary columns

    return lazy_df.collect()

In [8]:
data = add_search_price_features(data)

In [9]:
print(f"shape: {data.shape}")
print(f"data types: {data.dtypes}")
data.head(20)

shape: (18145372, 62)
data types: [Int64, Float64, Float64, Float64, Float64, Float64, Float64, Float64, Int64, String, Datetime(time_unit='ns', time_zone=None), String, Float64, Float64, Int32, UInt32, UInt32, UInt32, String, String, Int32, Int32, Int32, Int32, Float64, Float64, Int32, Int32, Int8, Int8, Int8, Int8, Int32, Int32, UInt32, String, String, UInt32, String, Float64, Float64, Float64, Float64, Float64, Float64, Float64, UInt32, Float64, UInt32, Float64, Float64, UInt32, Float64, Float64, Float64, Int32, Int32, Int32, Float64, Float64, Float64, Float64]


Id,miniRules0_monetaryAmount,miniRules0_percentage,miniRules0_statusInfos,miniRules1_monetaryAmount,miniRules1_percentage,miniRules1_statusInfos,pricingInfo_isAccessTP,pricingInfo_passengerCount,ranker_id,requestDate,searchRoute,taxes,totalPrice,is_roundtrip,leg0_num_segments,leg1_num_segments,total_segments,route_origin,route_destination,origin_is_major_hub,destination_is_major_hub,includes_major_hub,booking_lead_time_min,leg0_duration_min,leg1_duration_min,leg0_flight_time_min,leg1_flight_time_min,leg0_departure_hour,leg0_arrival_hour,leg1_departure_hour,leg1_arrival_hour,has_cancellation_fee,has_exchange_fee,unique_carriers,primary_carrier,marketing_carrier,aircraft_diversity,primary_aircraft,max_cabin_class,avg_cabin_class,min_seats_available,total_seats_available,baggage_allowance_quantity,baggage_allowance_weight,total_duration,route_popularity,route_popularity_log,price_rank,price_percentile,price_zscore,options_count,price_ratio_to_min,price_ratio_to_max,price_ratio_to_median,is_cheapest,is_most_expensive,is_below_median,price_range,price_std,price_cv,price_gap_to_next_cheapest
i64,f64,f64,f64,f64,f64,f64,f64,i64,str,datetime[ns],str,f64,f64,i32,u32,u32,u32,str,str,i32,i32,i32,i32,f64,f64,i32,i32,i8,i8,i8,i8,i32,i32,u32,str,str,u32,str,f64,f64,f64,f64,f64,f64,f64,u32,f64,u32,f64,f64,u32,f64,f64,f64,i32,i32,i32,f64,f64,f64,f64
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",370.0,16884.0,1,1,1,2,"""TLK""","""KJA""",0,0,0,42516,160.0,155.0,160,155,15,16,9,14,0,0,1,"""KV""","""KV""",1,"""YK2""",1.0,1.0,9.0,18.0,1.0,0.0,315.0,25,3.258097,1,0.0,-2.605417,25,1.0,0.196166,0.314443,1,0,1,69186.0,18905.2881,0.285837,34241.0
1,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",2240.0,51125.0,1,2,2,4,"""TLK""","""KJA""",0,0,0,42141,445.0,505.0,250,245,9,14,22,8,1,1,1,"""S7""","""S7""",1,"""E70""",1.0,1.0,4.0,26.0,1.0,0.0,950.0,25,3.258097,2,4.166667,-0.794231,25,3.028015,0.593993,0.952137,0,0,1,69186.0,18905.2881,0.285837,0.0
2,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",2240.0,53695.0,1,2,2,4,"""TLK""","""KJA""",0,0,0,42141,445.0,505.0,250,245,9,14,22,8,1,1,1,"""S7""","""S7""",1,"""E70""",1.0,1.0,4.0,26.0,1.0,0.0,950.0,25,3.258097,8,29.166667,-0.65829,25,3.18023,0.623853,1.0,0,0,0,69186.0,18905.2881,0.285837,-2570.0
3,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",2240.0,81880.0,1,2,2,4,"""TLK""","""KJA""",0,0,0,42141,445.0,505.0,250,245,9,14,22,8,0,0,1,"""S7""","""S7""",1,"""E70""",1.0,1.0,4.0,26.0,1.0,0.0,950.0,25,3.258097,14,54.166667,0.832563,25,4.849562,0.951319,1.524909,0,0,0,69186.0,18905.2881,0.285837,-30755.0
4,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",2240.0,86070.0,1,2,2,4,"""TLK""","""KJA""",0,0,0,42141,445.0,505.0,250,245,9,14,22,8,0,0,1,"""S7""","""S7""",1,"""E70""",1.0,1.0,4.0,26.0,1.0,0.0,950.0,25,3.258097,20,79.166667,1.054194,25,5.097726,1.0,1.602943,0,1,0,69186.0,18905.2881,0.285837,-34945.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
15,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",2240.0,81880.0,1,2,2,4,"""TLK""","""KJA""",0,0,0,42141,445.0,755.0,250,245,9,14,17,8,0,0,1,"""S7""","""S7""",1,"""E70""",1.0,1.0,4.0,26.0,1.0,0.0,1200.0,25,3.258097,14,54.166667,0.832563,25,4.849562,0.951319,1.524909,0,0,0,69186.0,18905.2881,0.285837,0.0
16,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",2240.0,86070.0,1,2,2,4,"""TLK""","""KJA""",0,0,0,42141,445.0,755.0,250,245,9,14,17,8,0,0,1,"""S7""","""S7""",1,"""E70""",1.0,1.0,4.0,26.0,1.0,0.0,1200.0,25,3.258097,20,79.166667,1.054194,25,5.097726,1.0,1.602943,0,1,0,69186.0,18905

In [10]:
def camel_to_snake(name):
    """Convert camelCase or PascalCase to snake_case"""
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    s2 = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1)
    return s2.lower()

def convert_columns_to_snake_case(df):
    """Convert all column names in a polars DataFrame to snake_case"""
    return df.rename({col: camel_to_snake(col) for col in df.columns})

In [11]:
data = convert_columns_to_snake_case(data)
data.head(100)

id,mini_rules0_monetary_amount,mini_rules0_percentage,mini_rules0_status_infos,mini_rules1_monetary_amount,mini_rules1_percentage,mini_rules1_status_infos,pricing_info_is_access_tp,pricing_info_passenger_count,ranker_id,request_date,search_route,taxes,total_price,is_roundtrip,leg0_num_segments,leg1_num_segments,total_segments,route_origin,route_destination,origin_is_major_hub,destination_is_major_hub,includes_major_hub,booking_lead_time_min,leg0_duration_min,leg1_duration_min,leg0_flight_time_min,leg1_flight_time_min,leg0_departure_hour,leg0_arrival_hour,leg1_departure_hour,leg1_arrival_hour,has_cancellation_fee,has_exchange_fee,unique_carriers,primary_carrier,marketing_carrier,aircraft_diversity,primary_aircraft,max_cabin_class,avg_cabin_class,min_seats_available,total_seats_available,baggage_allowance_quantity,baggage_allowance_weight,total_duration,route_popularity,route_popularity_log,price_rank,price_percentile,price_zscore,options_count,price_ratio_to_min,price_ratio_to_max,price_ratio_to_median,is_cheapest,is_most_expensive,is_below_median,price_range,price_std,price_cv,price_gap_to_next_cheapest
i64,f64,f64,f64,f64,f64,f64,f64,i64,str,datetime[ns],str,f64,f64,i32,u32,u32,u32,str,str,i32,i32,i32,i32,f64,f64,i32,i32,i8,i8,i8,i8,i32,i32,u32,str,str,u32,str,f64,f64,f64,f64,f64,f64,f64,u32,f64,u32,f64,f64,u32,f64,f64,f64,i32,i32,i32,f64,f64,f64,f64
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",370.0,16884.0,1,1,1,2,"""TLK""","""KJA""",0,0,0,42516,160.0,155.0,160,155,15,16,9,14,0,0,1,"""KV""","""KV""",1,"""YK2""",1.0,1.0,9.0,18.0,1.0,0.0,315.0,25,3.258097,1,0.0,-2.605417,25,1.0,0.196166,0.314443,1,0,1,69186.0,18905.2881,0.285837,34241.0
1,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",2240.0,51125.0,1,2,2,4,"""TLK""","""KJA""",0,0,0,42141,445.0,505.0,250,245,9,14,22,8,1,1,1,"""S7""","""S7""",1,"""E70""",1.0,1.0,4.0,26.0,1.0,0.0,950.0,25,3.258097,2,4.166667,-0.794231,25,3.028015,0.593993,0.952137,0,0,1,69186.0,18905.2881,0.285837,0.0
2,2300.0,0.0,1.0,3500.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",2240.0,53695.0,1,2,2,4,"""TLK""","""KJA""",0,0,0,42141,445.0,505.0,250,245,9,14,22,8,1,1,1,"""S7""","""S7""",1,"""E70""",1.0,1.0,4.0,26.0,1.0,0.0,950.0,25,3.258097,8,29.166667,-0.65829,25,3.18023,0.623853,1.0,0,0,0,69186.0,18905.2881,0.285837,-2570.0
3,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",2240.0,81880.0,1,2,2,4,"""TLK""","""KJA""",0,0,0,42141,445.0,505.0,250,245,9,14,22,8,0,0,1,"""S7""","""S7""",1,"""E70""",1.0,1.0,4.0,26.0,1.0,0.0,950.0,25,3.258097,14,54.166667,0.832563,25,4.849562,0.951319,1.524909,0,0,0,69186.0,18905.2881,0.285837,-30755.0
4,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1,"""98ce0dabf6964640b63079fbafd42c…",2024-05-17 03:03:08,"""TLKKJA/KJATLK""",2240.0,86070.0,1,2,2,4,"""TLK""","""KJA""",0,0,0,42141,445.0,505.0,250,245,9,14,22,8,0,0,1,"""S7""","""S7""",1,"""E70""",1.0,1.0,4.0,26.0,1.0,0.0,950.0,25,3.258097,20,79.166667,1.054194,25,5.097726,1.0,1.602943,0,1,0,69186.0,18905.2881,0.285837,-34945.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
95,2300.0,0.0,1.0,3500.0,0.0,1.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",2024-05-17 03:40:53,"""KHVUUS""",1235.0,89570.0,0,2,0,2,"""KHV""","""UUS""",0,0,0,4584,1410.0,0.0,645,0,8,8,0,0,1,1,1,"""S7""","""S7""",1,"""321""",2.0,1.5,3.0,12.0,1.0,0.0,1410.0,291,5.676754,26,75.757576,0.75143,34,12.361303,0.536765,2.543482,0,0,0,159624.0,46232.122167,0.843193,3305.0
96,4000.0,0.0,1.0,0.0,0.0,0.0,0.0,1,"""e04b757602824a4dbe227f1e67dbdb…",2024-05-17 03:40:53,"""KHVUUS""",1235.0,92875.0,0,2,0,2,"""KHV""","""UUS""",0,0,0,4584,1410.0,0.0,645,0,8,8,0,0,1,0,1,"""S7""","""S7""",1,"""321""",2.0,2.0,3.0,6.0,1.0,0.0,1410.0,291,5.676754,27,78.787879,0.822917,34,12.817417,0.556571,2.637333,0,0,0

In [12]:
def prepare_features_for_clustering(df):
    # Features to drop
    drop_cols = ['id', 'ranker_id', 'request_date', 'search_route']

    # Identify categorical features (non-numeric columns)
    categorical_features = [col for col, dtype in df.schema.items() if not dtype.is_numeric()]

    # Using lazy computations to avoid memory issues
    lazy_df = df.lazy()

    # Encode categorical features
    encoding_expressions = []
    encoders = {}  # Mapping of feature name to encoding dictionary so we can evaluate cluster assignments

    for feature in categorical_features:
        # Get unique values for the feature
        unique_values = df[feature].fill_null('MISSING').unique().sort().to_list()

        # Create mapping dictionary
        encoders[feature] = {val: idx for idx, val in enumerate(unique_values)}

        # Convert to categorical then to physical (integer codes)
        encoding_expr = (
            pl.col(feature)
            .fill_null('MISSING')
            .cast(pl.Categorical)
            .to_physical()
            .alias(f'{feature}_encoded')
        )
        encoding_expressions.append(encoding_expr)

    # Apply encodings and exclude original categorical features
    result_df = lazy_df.with_columns(encoding_expressions).select(pl.exclude(categorical_features + drop_cols)).fill_null(0).collect()

    return result_df, encoders

In [13]:
# Encode categorical features
clustering_df, encoders = prepare_features_for_clustering(data)

# Convert to Pandas DataFrame for StandardScaler
clustering_pd_df = clustering_df.to_pandas().fillna(0)

In [14]:
# Scale the features for use with KMeans
scaler = StandardScaler()
features_scaled = scaler.fit_transform(clustering_pd_df)

del clustering_pd_df
gc.collect()

64

In [15]:
# Use MiniBatchKMeans to cluster the data (more efficient for very large datasets)
kmeans = MiniBatchKMeans(n_clusters=30, batch_size=50000, random_state=42)
clusters = kmeans.fit_predict(features_scaled)

In [22]:
def interpret_clusters(original_df, clusters, n_clusters_to_show=10):
    # Add clusters to Polars dataframe
    df_with_clusters = original_df.with_columns([
        pl.Series("cluster", clusters)
    ])

    print(f"=== Cluster Analysis ===")
    print(f"Total clusters: {len(np.unique(clusters))}")
    print(f"Dataset size: {len(df_with_clusters):,} rows")

    for cluster_id in range(min(n_clusters_to_show, len(np.unique(clusters)))):
        # Filter to cluster using Polars
        cluster_data = df_with_clusters.filter(pl.col("cluster") == cluster_id)
        cluster_size = len(cluster_data)

        print(f"\n{'='*50}")
        print(f"CLUSTER {cluster_id} ({cluster_size:,} flights, {cluster_size/len(df_with_clusters)*100:.1f}%)")
        print(f"{'='*50}")

        # Show top categorical features using Polars operations
        if 'search_route' in cluster_data.columns:
            print(f"\nTop Routes:")
            top_routes = cluster_data.group_by('search_route').agg(pl.len().alias('count')).sort('count', descending=True).head()
            for row in top_routes.iter_rows(named=True):
                route, count = row['search_route'], row['count']
                pct = count/cluster_size*100
                print(f"  {route}: {count:,} ({pct:.1f}%)")

        if 'primary_carrier' in cluster_data.columns:
            print(f"\nTop Carriers:")
            top_carriers = cluster_data.group_by('primary_carrier').agg(pl.len().alias('count')).sort('count', descending=True).head()
            for row in top_carriers.iter_rows(named=True):
                carrier, count = row['primary_carrier'], row['count']
                pct = count/cluster_size*100
                print(f"  {carrier}: {count:,} ({pct:.1f}%)")

        # Numerical statistics using Polars
        print(f"\nFlight Characteristics:")
        stats = cluster_data.select([
            pl.col('total_price').mean().alias('avg_price'),
            pl.col('total_price').median().alias('med_price'),
            pl.col('total_duration').mean().alias('avg_duration'),
            pl.col('total_segments').mean().alias('avg_segments'),
            pl.col('is_roundtrip').mean().alias('roundtrip_pct')
        ]).to_pandas().iloc[0]

        print(f"  Avg Price: ${stats['avg_price']:,.0f} (${stats['med_price']:,.0f} median)")
        print(f"  Avg Duration: {stats['avg_duration']:.0f} min ({stats['avg_duration']/60:.1f} hrs)")
        print(f"  Avg Segments: {stats['avg_segments']:.1f}")
        print(f"  Roundtrip: {stats['roundtrip_pct']*100:.1f}%")


In [23]:
interpret_clusters(data, clusters, n_clusters_to_show=10)


=== Cluster Analysis ===
Total clusters: 30
Dataset size: 18,145,372 rows

CLUSTER 0 (1,231,022 flights, 6.8%)

Top Routes:
  MOWAER/AERMOW: 219,045 (17.8%)
  LEDMOW/MOWLED: 123,356 (10.0%)
  MOWSVX/SVXMOW: 101,697 (8.3%)
  AERMOW/MOWAER: 93,442 (7.6%)
  MOWLED/LEDMOW: 82,685 (6.7%)

Top Carriers:
  SU: 986,866 (80.2%)
  U6: 136,022 (11.0%)
  S7: 98,700 (8.0%)
  DP: 2,605 (0.2%)
  UT: 2,482 (0.2%)

Flight Characteristics:
  Avg Price: $36,733 ($31,940 median)
  Avg Duration: 324 min (5.4 hrs)
  Avg Segments: 2.0
  Roundtrip: 100.0%

CLUSTER 1 (286,178 flights, 1.6%)

Top Routes:
  MOWSHA/SHAMOW: 36,286 (12.7%)
  MOWPAR/PARMOW: 28,380 (9.9%)
  MOWCAN/CANMOW: 18,593 (6.5%)
  MOWVCE/VCEMOW: 16,761 (5.9%)
  MOWTAS/TASMOW: 13,518 (4.7%)

Top Carriers:
  TK: 121,554 (42.5%)
  EK: 64,737 (22.6%)
  QR: 20,626 (7.2%)
  MU: 20,568 (7.2%)
  CA: 16,752 (5.9%)

Flight Characteristics:
  Avg Price: $159,229 ($130,940 median)
  Avg Duration: 2514 min (41.9 hrs)
  Avg Segments: 4.1
  Roundtrip: 96.3%


In [ ]:
class FlightSelectionPredictor:
    def __init__(self, n_flight_clusters=20, n_customer_clusters=10):
        self.n_flight_clusters = n_flight_clusters
        self.n_customer_clusters = n_customer_clusters

        # Clustering models
        self.flight_clusterer = KMeans(n_clusters=n_flight_clusters, random_state=42)
        self.customer_clusterer = KMeans(n_clusters=n_customer_clusters, random_state=42)

        # Scalers
        self.flight_scaler = StandardScaler()
        self.customer_scaler = StandardScaler()

        # Prediction model
        self.predictor = GradientBoostingClassifier(n_estimators=200, random_state=42)

        # Label encoders for categorical variables
        self.label_encoders = {}

    def extract_flight_features(self, df):
        """Extract and engineer flight-specific features"""
        features = pd.DataFrame(index=df.index)

        # Basic flight info
        features['total_price'] = df['totalPrice']
        features['taxes'] = df['taxes']
        features['passenger_count'] = df['pricingInfo_passengerCount']

        # Duration features (convert to minutes)
        features['leg0_duration_min'] = pd.to_timedelta(df['legs0_duration']).dt.total_seconds() / 60
        features['leg1_duration_min'] = pd.to_timedelta(df['legs1_duration']).dt.total_seconds() / 60
        features['total_duration'] = features['leg0_duration_min'] + features['leg1_duration_min']

        # Time features
        features['leg0_departure_hour'] = pd.to_datetime(df['legs0_departureAt']).dt.hour
        features['leg0_arrival_hour'] = pd.to_datetime(df['legs0_arrivalAt']).dt.hour
        features['leg1_departure_hour'] = pd.to_datetime(df['legs1_departureAt']).dt.hour
        features['leg1_arrival_hour'] = pd.to_datetime(df['legs1_arrivalAt']).dt.hour

        # Number of segments (connections)
        features['leg0_segments'] = (~df['legs0_segments1_aircraft_code'].isna()).astype(int) + 1
        features['leg1_segments'] = (~df['legs1_segments1_aircraft_code'].isna()).astype(int) + 1
        features['total_segments'] = features['leg0_segments'] + features['leg1_segments']

        # Carrier diversity (different airlines)
        carriers = []
        for i in range(4):  # up to 4 segments per leg
            for leg in ['legs0', 'legs1']:
                carrier_col = f'{leg}_segments{i}_marketingCarrier_code'
                if carrier_col in df.columns:
                    carriers.append(df[carrier_col])

        if carriers:
            carrier_df = pd.concat(carriers, axis=1)
            features['unique_carriers'] = carrier_df.nunique(axis=1)
        else:
            features['unique_carriers'] = 1

        # Seat availability (proxy for popularity/scarcity)
        seat_cols = [col for col in df.columns if 'seatsAvailable' in col]
        if seat_cols:
            features['min_seats_available'] = df[seat_cols].min(axis=1)
            features['total_seats_available'] = df[seat_cols].sum(axis=1)

        # Baggage allowance
        baggage_cols = [col for col in df.columns if 'baggageAllowance_quantity' in col]
        if baggage_cols:
            features['total_baggage'] = df[baggage_cols].sum(axis=1)

        # Cabin class (assuming 1=economy, higher=premium)
        cabin_cols = [col for col in df.columns if 'cabinClass' in col]
        if cabin_cols:
            features['max_cabin_class'] = df[cabin_cols].max(axis=1)
            features['avg_cabin_class'] = df[cabin_cols].mean(axis=1)

        # Pricing rules complexity
        features['has_cancellation_fee'] = (df['miniRules0_monetaryAmount'] > 0).astype(int)
        features['has_change_fee'] = (df['miniRules1_monetaryAmount'] > 0).astype(int)

        # Fill missing values
        features = features.fillna(0)

        return features

    def extract_customer_features(self, df):
        """Extract customer-specific features"""
        features = pd.DataFrame(index=df.index)

        # Basic customer info
        features['by_self'] = df['bySelf'].astype(int)
        features['frequent_flyer'] = (~df['frequentFlyer'].isna()).astype(int)
        features['nationality'] = df['nationality']
        features['is_access_3d'] = df['isAccess3D'].astype(int)
        features['is_vip'] = df['isVip'].astype(int)
        features['sex'] = df['sex'].astype(int)
        features['company_id'] = df['companyID'].fillna(-1)
        features['has_corporate_tariff'] = (~df['corporateTariffCode'].isna()).astype(int)

        # Request timing
        features['request_date'] = pd.to_datetime(df['requestDate'])
        features['request_hour'] = features['request_date'].dt.hour
        features['request_day_of_week'] = features['request_date'].dt.dayofweek
        features['days_in_advance'] = (pd.to_datetime(df['legs0_departureAt']) - features['request_date']).dt.days

        # Route complexity
        features['search_route'] = df['searchRoute']

        return features.drop('request_date', axis=1)

    def fit_flight_clusters(self, flight_features):
        """Cluster flights based on their characteristics"""
        # Scale features
        scaled_features = self.flight_scaler.fit_transform(flight_features)

        # Fit clustering
        flight_clusters = self.flight_clusterer.fit_predict(scaled_features)

        return flight_clusters

    def fit_customer_clusters(self, customer_features):
        """Cluster customers based on their characteristics"""
        # Encode categorical variables
        encoded_features = customer_features.copy()

        categorical_cols = ['search_route']
        for col in categorical_cols:
            if col in encoded_features.columns:
                if col not in self.label_encoders:
                    self.label_encoders[col] = LabelEncoder()
                encoded_features[col] = self.label_encoders[col].fit_transform(encoded_features[col].astype(str))

        # Scale features
        scaled_features = self.customer_scaler.fit_transform(encoded_features)

        # Fit clustering
        customer_clusters = self.customer_clusterer.fit_predict(scaled_features)

        return customer_clusters

    def create_customer_flight_preferences(self, df, customer_clusters, flight_clusters):
        """Create features representing customer preferences within flight clusters"""
        preference_features = pd.DataFrame(index=df.index)

        # Group by customer cluster and calculate flight cluster preferences
        for customer_cluster in range(self.n_customer_clusters):
            customer_mask = customer_clusters == customer_cluster
            customer_data = df[customer_mask]

            # Calculate selection rates for each flight cluster within this customer cluster
            for flight_cluster in range(self.n_flight_clusters):
                flight_mask = flight_clusters[customer_mask] == flight_cluster

                if flight_mask.sum() > 0:
                    # This would need actual selection data - for now, using price as proxy
                    selection_rate = 1.0 / (1.0 + customer_data[flight_mask]['totalPrice'].mean() / 10000)
                else:
                    selection_rate = 0.0

                preference_features[f'customer_cluster_{customer_cluster}_flight_cluster_{flight_cluster}_pref'] = \
                    ((customer_clusters == customer_cluster) & (flight_clusters == flight_cluster)).astype(float) * selection_rate

        return preference_features

    def fit(self, df, selections=None):
        """Fit the complete model"""
        print("Extracting flight features...")
        flight_features = self.extract_flight_features(df)

        print("Extracting customer features...")
        customer_features = self.extract_customer_features(df)

        print("Clustering flights...")
        flight_clusters = self.fit_flight_clusters(flight_features)

        print("Clustering customers...")
        customer_clusters = self.fit_customer_clusters(customer_features)

        print("Creating preference features...")
        preference_features = self.create_customer_flight_preferences(df, customer_clusters, flight_clusters)

        # Combine all features
        all_features = pd.concat([
            flight_features,
            customer_features,
            preference_features,
            pd.Series(flight_clusters, name='flight_cluster', index=df.index),
            pd.Series(customer_clusters, name='customer_cluster', index=df.index)
        ], axis=1)

        # If we have selection data, train the predictor
        if selections is not None:
            print("Training prediction model...")
            # For now, create synthetic selection data based on price (lower price = higher selection probability)
            # In practice, you'd use your actual selection data
            y = (df['totalPrice'] < df['totalPrice'].median()).astype(int)

            X_train, X_test, y_train, y_test = train_test_split(
                all_features, y, test_size=0.2, random_state=42, stratify=y
            )

            self.predictor.fit(X_train, y_train)

            # Evaluate
            train_score = self.predictor.score(X_train, y_train)
            test_score = self.predictor.score(X_test, y_test)

            print(f"Training accuracy: {train_score:.3f}")
            print(f"Test accuracy: {test_score:.3f}")

            # Feature importance
            feature_importance = pd.DataFrame({
                'feature': all_features.columns,
                'importance': self.predictor.feature_importances_
            }).sort_values('importance', ascending=False)

            print("\nTop 10 most important features:")
            print(feature_importance.head(10))

        self.is_fitted = True
        return self

    def predict_probabilities(self, df):
        """Predict selection probabilities for flights"""
        if not hasattr(self, 'is_fitted'):
            raise ValueError("Model must be fitted before making predictions")

        # Extract features using the same process as training
        flight_features = self.extract_flight_features(df)
        customer_features = self.extract_customer_features(df)

        # Apply clustering
        scaled_flight_features = self.flight_scaler.transform(flight_features)
        flight_clusters = self.flight_clusterer.predict(scaled_flight_features)

        # Encode categorical variables in customer features
        encoded_customer_features = customer_features.copy()
        for col, encoder in self.label_encoders.items():
            if col in encoded_customer_features.columns:
                encoded_customer_features[col] = encoder.transform(encoded_customer_features[col].astype(str))

        scaled_customer_features = self.customer_scaler.transform(encoded_customer_features)
        customer_clusters = self.customer_clusterer.predict(scaled_customer_features)

        # Create preference features
        preference_features = self.create_customer_flight_preferences(df, customer_clusters, flight_clusters)

        # Combine features
        all_features = pd.concat([
            flight_features,
            customer_features,
            preference_features,
            pd.Series(flight_clusters, name='flight_cluster', index=df.index),
            pd.Series(customer_clusters, name='customer_cluster', index=df.index)
        ], axis=1)

        # Predict probabilities
        probabilities = self.predictor.predict_proba(all_features)[:, 1]

        return probabilities

# Example usage
def example_usage():
    """Example of how to use the FlightSelectionPredictor"""

    # Load your data
    # df = pd.read_csv('your_flight_data.csv')

    # Initialize and fit the model
    predictor = FlightSelectionPredictor(n_flight_clusters=15, n_customer_clusters=8)

    # Fit the model (you'd pass actual selection data here)
    # predictor.fit(df, selections=your_selection_data)

    # Make predictions for new flight options
    # probabilities = predictor.predict_probabilities(new_flight_options)

    print("Model setup complete. Use predictor.fit() with your data to train the model.")

if __name__ == "__main__":
    example_usage()